In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##Clustering##
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import (KMeans, AgglomerativeClustering)
from scipy.cluster.hierarchy import (dendrogram , cut_tree)
from ISLP.cluster import compute_linkage

# Clustering

Lets apply hirerical clustering and pca to the data and see if we can find the group of migrator starts by this approach. 

In [4]:
data = pd.read_csv('Faellesdata_cleaned.csv')
scaler = StandardScaler(with_std=True, with_mean=True)
data_scaled = scaler.fit_transform(data)

In [ ]:
HClust = AgglomerativeClustering 
hc_comp = HClust(distance_threshold=0, n_clusters=None, linkage='complete')
hc_comp.fit(data_scaled)

cargs = {'color_threshold':-np.inf, 'above_threshold_color':'black'} #to avoid precoloring the dendrogram
linkage_comp = compute_linkage(hc_comp)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
dendrogram(linkage_comp, ax=ax, **cargs);

s